## Modeling for Fraud Detection

### 1. Fraud Data Modeling

In [6]:
import sys
sys.path.append("..")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix,
    precision_recall_curve, average_precision_score, f1_score, recall_score, precision_score
)
from src.features import resample_smote, resample_undersample

%matplotlib inline

data_path = Path("../data/processed")
fraud_df = pd.read_csv(data_path / "fraud_data_processed.csv")
print("Fraud data shape:", fraud_df.shape)
fraud_df.head()

Fraud data shape: (151112, 195)


,user_id,purchase_value,age,class,purchase_hour,purchase_dayofweek,time_since_signup_hours,source_Direct,source_SEO,browser_FireFox,...,country_Unknown,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,62421,0.494721,0.331793,0,10,6,-1.013679,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,173212,-0.214781,-0.364448,0,17,4,-1.230613,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,242286,-0.214781,-0.132367,0,8,1,-1.337931,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,370003,-0.214781,0.795954,0,21,3,0.800513,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,119824,0.985915,0.563874,0,7,6,-1.127359,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
# Target
y = fraud_df['class']
# Features: drop user_id (identifier) and target
X = fraud_df.drop(columns=['class', 'user_id'])

print("X shape:", X.shape)
print("y distribution:\n", y.value_counts())

X shape: (151112, 193)
y distribution:
 class
0    136961
1     14151
Name: count, dtype: int64


In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Train fraud rate: {y_train.mean():.3%}")
print(f"Test fraud rate: {y_test.mean():.3%}")

Train: (120889, 193), Test: (30223, 193)
Train fraud rate: 9.365%
Test fraud rate: 9.364%


In [4]:
X_train_res, y_train_res = resample_smote(X_train, y_train)
print("After SMOTE:", X_train_res.shape, y_train_res.shape)
print("Resampled class distribution:\n", pd.Series(y_train_res).value_counts())

After SMOTE: (219136, 193) (219136,)
Resampled class distribution:
 class
0    109568
1    109568
Name: count, dtype: int64


In [5]:
# Baseline Logistic Regression
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_res, y_train_res)

# Predict on test set (unseen, not resampled)
y_pred = lr.predict(X_test)
y_prob = lr.predict_proba(X_test)[:, 1]

# Evaluation
print("=== Logistic Regression ===")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print(f"Average Precision: {average_precision_score(y_test, y_prob):.4f}")

=== Logistic Regression ===
              precision    recall  f1-score   support

           0       0.95      0.66      0.78     27393
           1       0.17      0.70      0.28      2830

    accuracy                           0.66     30223
   macro avg       0.56      0.68      0.53     30223
weighted avg       0.88      0.66      0.73     30223

Confusion Matrix:
[[17966  9427]
 [  851  1979]]
Average Precision: 0.3798


In [7]:
# Random Forest with SMOTE training data
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train_res, y_train_res)

y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]

print("=== Random Forest ===")
print(classification_report(y_test, y_pred_rf))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))
print(f"Average Precision: {average_precision_score(y_test, y_prob_rf):.4f}")

=== Random Forest ===
              precision    recall  f1-score   support

           0       0.95      1.00      0.98     27393
           1       0.99      0.52      0.69      2830

    accuracy                           0.96     30223
   macro avg       0.97      0.76      0.83     30223
weighted avg       0.96      0.96      0.95     30223

Confusion Matrix:
[[27379    14]
 [ 1346  1484]]
Average Precision: 0.6222


In [8]:
# Comparison table
results = {
    'Model': ['Logistic Regression', 'Random Forest'],
    'AP': [
        average_precision_score(y_test, y_prob),
        average_precision_score(y_test, y_prob_rf)
    ],
    'F1_Fraud': [
        f1_score(y_test, y_pred, pos_label=1),
        f1_score(y_test, y_pred_rf, pos_label=1)
    ],
    'Recall_Fraud': [
        recall_score(y_test, y_pred, pos_label=1),
        recall_score(y_test, y_pred_rf, pos_label=1)
    ],
    'Precision_Fraud': [
        precision_score(y_test, y_pred, pos_label=1),
        precision_score(y_test, y_pred_rf, pos_label=1)
    ]
}
results_df = pd.DataFrame(results)
results_df

,Model,AP,F1_Fraud,Recall_Fraud,Precision_Fraud
0,Logistic Regression,0.379848,0.278028,0.699293,0.173505
1,Random Forest,0.622244,0.685767,0.524382,0.990654


In [9]:
# Undersample majority class
X_train_under, y_train_under = resample_undersample(X_train, y_train)
print("After undersampling:", X_train_under.shape)

# Train RF on undersampled data
rf_under = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_under.fit(X_train_under, y_train_under)

y_pred_under = rf_under.predict(X_test)
y_prob_under = rf_under.predict_proba(X_test)[:, 1]

print("=== Random Forest (Undersampled) ===")
print(classification_report(y_test, y_pred_under))
print(f"Average Precision: {average_precision_score(y_test, y_prob_under):.4f}")

After undersampling: (22642, 193)
=== Random Forest (Undersampled) ===
              precision    recall  f1-score   support

           0       0.95      1.00      0.98     27393
           1       0.99      0.52      0.69      2830

    accuracy                           0.96     30223
   macro avg       0.97      0.76      0.83     30223
weighted avg       0.96      0.96      0.95     30223

Average Precision: 0.6270
